In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, when, isnan, split
import pandas as pd
from pyspark.ml.feature import VectorAssembler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
spSession = SparkSession.builder.master("local").appName("Local-Session").getOrCreate()

In [3]:
# Loading files into a Data Frame

trainDF1 = spSession.read.csv("../dataset/train_3.csv", header = True) 
testDF1 = spSession.read.csv("../dataset/test_3.csv", header = True) 
type(trainDF1)

pyspark.sql.dataframe.DataFrame

In [4]:
# Converting string values to float

trainDF2 = trainDF1.select(*(col(c).cast("float").alias(c) for c in trainDF1.columns))
testDF2 = testDF1.select(*(col(c).cast("float").alias(c) for c in testDF1.columns)).orderBy("ID")

In [6]:
# Splitting data

X = trainDF2.drop("TARGET").toPandas()
Y = trainDF2.toPandas()["TARGET"]

test = testDF2.drop("ID").toPandas()

In [7]:
#Training Model - Logistic Regression - 0.72 Accuracy

model = LogisticRegression(max_iter= 10000)
fit = model.fit(X, Y)

result = pd.DataFrame(fit.predict(test), columns = ["TARGET"])
result.index.name = "ID"
ID = pd.read_csv("../dataset/test.csv")
result.index = ID["ID"]
result.to_csv("../dataset/Kaggle.csv")